
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [47]:
import numpy as np

"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):
    # Algoritmo de representación de punto flotante modificada.
    if x == np.inf or x == np.NINF or x == 0:
        return x
    
    signo = 1
    if x < 0:
        signo = -1
        x = abs(x)
    
    binario = ''
    parte_entera = int(x)
    parte_decimal = abs(x - parte_entera)
    
    while parte_entera > 0:
        div = divmod(parte_entera,2)
        parte_entera = div[0]
        
        if div[1] == 1:
            binario = str(1) + binario
        else:
            binario = str(0) + binario
            
    if parte_decimal == 0:
        binario = binario + '.0'
    elif x < 1:
        binario = binario +'0.'
    else:
        binario = binario + '.'

    bits_counter = 0
    while parte_decimal > 0 and bits_counter <= (2**bits_exp-2)/2+bits_mant-1:
        parte_decimal = 2*parte_decimal
        if parte_decimal >= 1:
            binario = binario + str(1)
            parte_decimal -= 1
        else:
            binario = binario + str(0)
        bits_counter += 1
            
    bits_lim = int(-1*((2**bits_exp-2)/2-1))
    if x < 2**bits_lim:
        if x < 2**bits_lim*2**-bits_mant:
            result = 0
        else:
            result = subnormal(binario, bits_mant, bits_lim)*signo
        
        return result
    else:
        binario = normalizar(binario)
    
    mantisa = binario[0][1:]
    exp = binario[1]
    if len(mantisa) > bits_mant + 1:
        aproximar = False
        if mantisa[bits_mant+1] == '0':
            mantisa = mantisa[:bits_mant+1]
        else:
            todos_cero = True
            b = bits_mant + 2
            while b < len(mantisa):
                if mantisa[b] == '1':
                    todos_cero = False
                    break
                b += 1
                
            if todos_cero:
                if mantisa[bits_mant] == '1':
                    aproximar = True
            else:
                aproximar = True
                
            mantisa = mantisa[:bits_mant+1]
            
        if aproximar:
            redondeo = redondear(mantisa)
            mantisa = redondeo[0]
            overflow = redondeo[1]
            
            if overflow:
                exp += 1
                
    final = (binario[0][0] + mantisa, exp)
    
    if final[1] > (2**bits_exp-2)/2:
        return np.inf*signo
    elif final[1] < -1*((2**bits_exp-2)/2-1):
        return 0

    result = evaluador(final[0],final[1])*signo
    return result

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    
    result = n1 + n2
    result = f_new_rep(result,bits_mant,bits_exp)
    
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    
    result = n1*n2
    result = f_new_rep(result,bits_mant,bits_exp)
    
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    n1 = f_new_rep(n1,bits_mant,bits_exp)
    n2 = f_new_rep(n2,bits_mant,bits_exp)
    
    result = n1/n2
    result = f_new_rep(result,bits_mant,bits_exp)
    
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    #Primera raiz es 1, encontrada de forma algebraica
    x_roots = np.array([1])
    
    #Usamos formulas para encontrar las otras 2 raices
    A_4_menos_A_2 = f_suma(A**4,-A**2,bits_mant,bits_exp)
    raiz_A_4_menos_A_2 = f_new_rep(np.sqrt(A_4_menos_A_2), bits_mant, bits_exp)
    
    r1 = f_suma(A**2, -raiz_A_4_menos_A_2, bits_mant, bits_exp)
    r1 = f_division(r1, A**2, bits_mant, bits_exp)
    
    r2 = f_suma(A**2, raiz_A_4_menos_A_2, bits_mant, bits_exp)
    r2 = f_division(r2, A**2, bits_mant, bits_exp)
    
    x_roots = np.append(x_roots,[r1,r2])
    x_roots = np.sort(x_roots)
    
    return x_roots


######################
#Funciones auxiliares#
######################
def evaluador(p_f, exp):
    val = 0
    pos = 0
    for x in p_f:
        if x == '.':
            continue
        val += int(x)*2**pos*2**exp
        pos -= 1
    return val

def normalizar(binario):
    exp = 0
    if binario[0] == '1':
        for x in range(len(binario)):
            if binario[x] == '.':
                binario = binario[0:x]+binario[x+1:]
                exp -= 1
                break
            exp +=1
        binario = binario[0] + '.' + binario[1:]
    else:
        for x in binario:
            if x == '.':
                continue
            elif x == '1':
                break
            
            exp -= 1
        binario = binario[abs(exp)+1:]
        binario = binario[0] + '.' + binario[1:]
        if len(binario) == 2:
            binario = binario + '0'
        
    return (binario,exp)
            
def subnormal(binario, mantisa, exp_fijo):
    binario = binario[0:2] + binario[abs(exp_fijo)+2:]
    indx = 0
    while indx < len(binario) and indx < mantisa + 2:
        indx+=1
    if indx < len(binario):
        binario = binario[:indx]
    
    result = 0
    exp = 0
    for x in binario:
        if x == '.':
            continue
        result += 2**exp_fijo*2**exp*int(x)
        exp -= 1
    
    return result

def redondear(mantisa):
    overflow = False
    indx = len(mantisa) - 1
    mantisa = list(mantisa)
    while indx > 0:
        if mantisa[indx] == '0':
            mantisa[indx] = '1'
            break
        mantisa[indx] = '0'
        indx -= 1
        
    if indx == 0:
        overflow = True
        
    return (''.join(mantisa),overflow)

### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\log(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


In [46]:
import numpy as np

"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""

def raiz_de_oreman(alpha, beta, tol = 1e-10):
    # Metodo de Newton basado en los apuntes del curso
    error_absoluto = np.inf
    x0 = 24.5
    
    f = lambda x: alpha*np.cos(x)+beta*np.sin(np.log(x))
    fp = lambda x :beta*np.cos(np.log(x))/x - alpha*np.sin(x)
    
    #Evaluamos en la derivada para elegir nuestro initial guess
    if fp(25) < 0:
        x0 = 25.5
    
    while error_absoluto > tol:
        x1 = x0 - f(x0)/fp(x0)
        
        if f(x1) == 0.0:
            break
        
        error_absoluto = abs(x0-x1)
    
        x0 = x1
    root = x1
    
    return root


# Referencias

C. Torres - Scientific Computing - https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1v2/03_roots_of_1D_equations.ipynb